First run the conversion script at convert.sh with the folder of your checkpoint.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


/n/home07/jchooi/.conda/envs/olmo2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
# Load the model and tokenizer
model_path = "/n/holyscratch01/sham_lab/summer_2024/checkpoints/test-batch-1024-39337249/step3000-unsharded"
model_path += "-hf"
olmo = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [27]:
olmo

OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(128, 16, padding_idx=7)
    (layers): ModuleList(
      (0-1): 2 x OlmoDecoderLayer(
        (self_attn): OlmoSdpaAttention(
          (q_proj): Linear(in_features=16, out_features=16, bias=False)
          (k_proj): Linear(in_features=16, out_features=16, bias=False)
          (v_proj): Linear(in_features=16, out_features=16, bias=False)
          (o_proj): Linear(in_features=16, out_features=16, bias=False)
          (rotary_emb): OlmoRotaryEmbedding()
        )
        (mlp): OlmoMLP(
          (gate_proj): Linear(in_features=16, out_features=8, bias=False)
          (up_proj): Linear(in_features=16, out_features=8, bias=False)
          (down_proj): Linear(in_features=8, out_features=16, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): OlmoLayerNorm()
        (post_attention_layernorm): OlmoLayerNorm()
      )
    )
    (norm): OlmoLayerNorm()
  )
  (lm_head): Linear(in_features=16, out_

In [3]:
# Prepare the input message
message = ["012"*100]
message[0] = message[0][:150]
inputs = tokenizer(message, return_tensors='pt', return_token_type_ids=False)
print(message[0])

012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012012


In [4]:
# Get the model's output logits
with torch.no_grad():
    outputs = olmo(**inputs)

logits = outputs.logits
print("Logits:", logits)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Logits: tensor([[[ 2.8174,  0.8746,  1.9126,  ..., -3.5451, -3.5116, -3.5138],
         [ 1.9858,  3.4932,  2.4783,  ..., -5.3553, -5.3961, -5.3758],
         [ 1.6352,  2.3408,  3.2565,  ..., -4.9417, -4.9648, -4.9276],
         ...,
         [ 2.9838,  2.3458,  3.0099,  ..., -5.5436, -5.5877, -5.5526],
         [ 2.1330,  3.1701,  2.9955,  ..., -5.6177, -5.6961, -5.6366],
         [ 2.3327,  2.5139,  3.3708,  ..., -5.5826, -5.6077, -5.5646]]])


In [5]:
logits.shape

torch.Size([1, 150, 128])

In [6]:
"""A bigram model (designed for evaluating against other models with KL divergence)"""
import numpy as np

class BigramModel:
    def __init__(self, dim):
        self.dim = dim
        # row_sum is the sum across columns for each row of freq_matrix
        # transition_matrix is freq_matrix / row_sum
        self.reset()
    
    def reset(self):
        self.row_sum = np.zeros(shape=(self.dim))
        # start with uniform distribution
        self.transition_matrix = np.full(shape=(self.dim, self.dim), fill_value=1/self.dim)
    
    def update(self, prev_token, current_token):
        # assume that the tokens are normalized to be in the range [0, dim)
        assert prev_token < self.dim, f"prev_token out of range. prev_token: {prev_token}, dim: {self.dim}"
        assert current_token < self.dim, f"current_token out of range. current_token: {current_token}, dim: {self.dim}"

        self.transition_matrix[prev_token] *= self.row_sum[prev_token] + self.dim
        self.transition_matrix[prev_token][current_token] += 1
        self.row_sum[prev_token] += 1
        self.transition_matrix[prev_token] /= self.row_sum[prev_token] + self.dim

    
    def get_transition_matrix(self):
        return self.transition_matrix


In [7]:
bg = BigramModel(3)

In [8]:
# train the bigram model

bg_input = [int(x) for x in message[0]]
for j in range(1, len(message[0])):
    bg.update(bg_input[j - 1], bg_input[j])

bg.get_transition_matrix()

array([[0.01886792, 0.96226415, 0.01886792],
       [0.01886792, 0.01886792, 0.96226415],
       [0.96153846, 0.01923077, 0.01923077]])

In [9]:
data = inputs['input_ids'][0]
data

tensor([0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2,
        0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2,
        0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2,
        0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2,
        0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2,
        0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2,
        0, 1, 2, 0, 1, 2])

In [10]:
import torch.nn.functional as F

bigram_probs = bg.get_transition_matrix()[data[-1]]
current_logits = logits[0][-1][:3]
q = F.log_softmax(current_logits, dim=0)
qexp = F.softmax(current_logits, dim=0)
p = torch.tensor(bigram_probs).to(q.device)
# reset the model for next Markov chain instance
F.kl_div(q, p, reduction="sum")

tensor(1.4008, dtype=torch.float64)

In [11]:
np.sum([p[i]*np.log(p[i]/qexp[i]) for i in range(len(p))])

1.400761408274022

In [12]:
print("model", qexp)
print("bigram", p)

model tensor([0.1991, 0.2387, 0.5622])
bigram tensor([0.9615, 0.0192, 0.0192], dtype=torch.float64)


In [13]:
message = ["012"*50]
message[0] = message[0][:149]
inputs = tokenizer(message, return_tensors='pt', return_token_type_ids=False)
print(inputs)
response = olmo.generate(**inputs, max_new_tokens=10, do_sample=False)
# response = olmo.generate(**inputs, max_new_tokens=10, do_sample=True, top_k=50, top_p=0.95)
print(response)
response_str = tokenizer.batch_decode(response, skip_special_tokens=True)[0]
print(len(response_str))
print("Input")
print(response_str[:len(message[0])])
print("Output")
print(response_str[len(message[0]):])

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (150). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


{'input_ids': tensor([[0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2,
         0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2,
         0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2,
         0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2,
         0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2,
         0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2,
         0, 1, 2, 0, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [14]:
import os
os.getcwd()

'/n/home07/jchooi/OLMo2/OLMo/1-setup-mc'

In [15]:
import sys
import os

# Add the target directory to the system path
scripts_dir = '/n/home07/jchooi/OLMo2/OLMo/scripts'
sys.path.append(scripts_dir)

from markov.generate_markov_chains import generate_markov_chains

chains = generate_markov_chains(num_seq=500, num_symbols=3, seq_len=150, deterministic=True)
chains[0]

tensor([1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2,
        1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2,
        1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2,
        1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2,
        1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2,
        1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2,
        1, 0, 2, 1, 0, 2])

In [16]:
sum_correct = 0

for chain in chains:
    inputs = [''.join([str(x) for x in chain[:-1].numpy()])]
    correct_response = str(chain[-1].item())
    inputs_tokenized = tokenizer(inputs, return_tensors='pt', return_token_type_ids=False)
    response = olmo.generate(**inputs_tokenized, max_new_tokens=1, do_sample=False)
    response_str = tokenizer.batch_decode(response, skip_special_tokens=True)[0][-1]
    # print("Input:", inputs)
    # print("correct_response:", correct_response)
    # print("response_str:", response_str)
    sum_correct += (correct_response == response_str)

print(sum_correct/len(chains))



0.156


Now let's try probabilistic Markov chains

In [17]:
chains = generate_markov_chains(num_seq=500, num_symbols=3, seq_len=150, deterministic=False)
chains[0]

tensor([0, 2, 1, 1, 1, 0, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 0, 2, 1, 1,
        1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 0, 2, 2, 1, 2, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 0, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 2, 1, 2, 1, 2, 2, 1, 1, 2, 0,
        2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 2,
        2, 1, 1, 1, 1, 1, 0, 2, 2, 0, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1])

In [18]:
sum_correct = 0

for chain in chains:
    inputs = [''.join([str(x) for x in chain[:-1].numpy()])]
    correct_response = str(chain[-1].item())
    inputs_tokenized = tokenizer(inputs, return_tensors='pt', return_token_type_ids=False)
    response = olmo.generate(**inputs_tokenized, max_new_tokens=1, do_sample=False)
    response_str = tokenizer.batch_decode(response, skip_special_tokens=True)[0][-1]
    # print("Input:", inputs)
    # print("correct_response:", correct_response)
    # print("response_str:", response_str)
    sum_correct += (correct_response == response_str)

print(sum_correct/len(chains))



0.424
